<a href="https://colab.research.google.com/github/yashshroff99/College-Projects/blob/main/ABC_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, date
from datetime import timedelta
from time import time
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Reading Inventory transactions data

In [ ]:
versa_sales = pd.read_csv('/content/drive/MyDrive/ABC_analysis/data_abc.csv')
versa_sales

,name,part_number,price,id,firm_id,delta,transaction_date,currency_id,currencies_name
0,R71329 SR 7,R71329 SR 7,6.55,136307,567,-1.0,2017-01-20,2,United States dollar
1,R71329 SR 7,R71329 SR 7,17.30,136307,567,-1.0,2017-01-20,2,United States dollar
2,R71329 SR 7,R71329 SR 7,18.85,136307,567,-1.0,2017-01-20,2,United States dollar
3,R71329 SR 7,R71329 SR 7,20.40,136307,567,-1.0,2017-01-20,2,United States dollar
4,R71391 SR 8,R71391 SR 8,6.00,135361,567,-1.0,2017-01-24,2,United States dollar
...,...,...,...,...,...,...,...,...,...
1796716,NaN,91000120,135.00,544287,570,1.0,2020-08-07,1,Canadian dollar
1796717,NaN,91000120,149.00,544287,570,1.0,2020-08-07,1,Canadian dollar
1796718,NaN,DC-012#3067#AEW02BD,67.00,387073,598,-4.0,2020-08-08,2,United States dollar
1796719,Bibs pacifier PEACH Size 2,200246,5.50,521256,609,138.0,2020-08-08,1,Canadian dollar


In [ ]:
versa_sales['transaction_date']=  pd.to_datetime(versa_sales['transaction_date'], errors = 'coerce')
versa_sales.head()

,name,part_number,price,id,firm_id,delta,transaction_date,currency_id,currencies_name
0,R71329 SR 7,R71329 SR 7,6.55,136307,567,-1.0,2017-01-20,2,United States dollar
1,R71329 SR 7,R71329 SR 7,17.30,136307,567,-1.0,2017-01-20,2,United States dollar
2,R71329 SR 7,R71329 SR 7,18.85,136307,567,-1.0,2017-01-20,2,United States dollar
3,R71329 SR 7,R71329 SR 7,20.40,136307,567,-1.0,2017-01-20,2,United States dollar
4,R71391 SR 8,R71391 SR 8,6.00,135361,567,-1.0,2017-01-24,2,United States dollar


In [ ]:
versa_sales_568 = versa_sales[(versa_sales["firm_id"]==568) & (versa_sales["delta"]<0)]
versa_sales_568

,name,part_number,price,id,firm_id,delta,transaction_date,currency_id,currencies_name
309,NaN,COTTAYBEI360,22.11,155810,568,-1.0,2017-03-16,2,United States dollar
310,NaN,COTTAYBEI360,65.00,155810,568,-1.0,2017-03-16,2,United States dollar
311,NaN,COTTAYBEI360,145.00,155810,568,-1.0,2017-03-16,2,United States dollar
312,NaN,COTTAYBEI360,116.00,155810,568,-1.0,2017-03-16,2,United States dollar
313,NaN,COTTAYBEI500,8.70,155826,568,-1.0,2017-03-16,2,United States dollar
...,...,...,...,...,...,...,...,...,...
1796609,NaN,COTPERSLT340,18.99,486970,568,-1.0,2020-08-07,2,United States dollar
1796663,NaN,LINCAMLIL300,88.25,507686,568,-1.0,2020-08-07,3,European Euro
1796664,NaN,LINCAMLIL300,360.00,507686,568,-1.0,2020-08-07,2,United States dollar
1796665,NaN,LINCAMLIL300,450.00,507686,568,-1.0,2020-08-07,2,United States dollar


In [ ]:
versa_sales_568.describe()

,price,id,firm_id,delta,currency_id
count,201454.000000,201454.000000,201454.0,201454.000000,201454.000000
mean,118.381073,277448.138662,568.0,-1.733741,2.070061
std,123.206278,136139.095416,0.0,3.870100,0.255250
min,0.000000,153724.000000,568.0,-569.000000,2.000000
25%,40.000000,157822.000000,568.0,-2.000000,2.000000
50%,85.000000,182552.000000,568.0,-1.000000,2.000000
75%,150.000000,383521.000000,568.0,-1.000000,2.000000
max,4100.000000,544135.000000,568.0,-1.000000,3.000000


In [ ]:
versa_sales_568["delta"] = versa_sales_568["delta"].abs()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
versa_sales_568 = versa_sales_568.drop(['firm_id'], axis=1)

In [ ]:
versa_sales_568['transaction_date'] = versa_sales_568.transaction_date.dt.to_period("M")

In [ ]:
versa_sales_568 = versa_sales_568.groupby(['id', 'part_number', 'price', 'currency_id','transaction_date' ], as_index=False).sum()

In [ ]:
versa_sales_568.delta.value_counts()

1.0      88319
2.0      34315
3.0      10781
4.0       6962
5.0       3181
         ...  
78.0         1
64.0         1
92.0         1
184.0        1
74.0         1
Name: delta, Length: 83, dtype: int64

#Obsolete items
Next step is to remove **obsolete items** first
Look at all items that have been in the system for 24 months or more, and then mark what we haven't sold any in those 24 months as obsolete.

In [ ]:
today = pd.to_datetime("today")
obsolete_time = (today - pd.DateOffset(years=2)).to_period("M")

In [ ]:
obsolete_items = versa_sales_568[versa_sales_568['transaction_date']<obsolete_time]
versa_sales_568 = versa_sales_568[versa_sales_568['transaction_date']>obsolete_time]

In [ ]:
obsolete_items

,id,part_number,price,currency_id,transaction_date,delta
0,153724,AAALPALIEBNONE,49.0,2,2019-01,1.0
3,153724,AAALPALIEBNONE,100.0,2,2019-01,1.0
6,153724,AAALPALIEBNONE,160.0,2,2019-01,1.0
9,153724,AAALPALIEBNONE,200.0,2,2019-01,1.0
12,153725,AAALPALIEBNONE,49.0,2,2017-03,1.0
...,...,...,...,...,...,...
118778,423902,COTINDMIX090,165.0,2,2019-02,1.0
118779,423903,COTINDMIX300,47.9,2,2019-02,1.0
118780,423903,COTINDMIX300,130.0,2,2019-02,1.0
118781,423903,COTINDMIX300,232.0,2,2019-02,1.0


#Non-stocked items
Remove **Non-stocked** items Look at all items that have been in the system for 12 months or more that we’ve only only sold them in 3 or less of those months as non-stocked.

In [ ]:
versa_sales_568[(versa_sales_568['transaction_date'] < (today - pd.DateOffset(years=1)).to_period("M"))]

,id,part_number,price,currency_id,transaction_date,delta
1,153724,AAALPALIEBNONE,49.00,2,2019-10,1.0
2,153724,AAALPALIEBNONE,49.00,2,2020-01,1.0
4,153724,AAALPALIEBNONE,100.00,2,2019-10,1.0
5,153724,AAALPALIEBNONE,100.00,2,2020-01,1.0
7,153724,AAALPALIEBNONE,160.00,2,2019-10,1.0
...,...,...,...,...,...,...
145775,510832,COTPERPOR400,275.00,2,2020-02,1.0
146354,513439,AALINJUNBLKMED,30.15,3,2020-02,1.0
146355,513439,AALINJUNBLKMED,65.00,2,2020-02,1.0
146356,513439,AALINJUNBLKMED,88.00,2,2020-02,1.0


In [ ]:
versa_sales_568['id'].count()

51393

In [ ]:
from google.cloud import bigquery

In [ ]:
(versa_sales_568.groupby(versa_sales_568['id']).sum().delta > 3)

id
153724    True
153739    True
153750    True
153753    True
153760    True
          ... 
544131    True
544132    True
544133    True
544134    True
544135    True
Name: delta, Length: 9648, dtype: bool

In [ ]:
versa_sales_568 = versa_sales_568[versa_sales_568['transaction_date']>'2019-01-01']
versa_sales_568

,id,part_number,price,currency_id,transaction_date,delta
1,153724,AAALPALIEBNONE,49.0,2,2019-10,1.0
2,153724,AAALPALIEBNONE,49.0,2,2020-01,1.0
4,153724,AAALPALIEBNONE,100.0,2,2019-10,1.0
5,153724,AAALPALIEBNONE,100.0,2,2020-01,1.0
7,153724,AAALPALIEBNONE,160.0,2,2019-10,1.0
...,...,...,...,...,...,...
153458,544134,ALPLIAPOP800,175.0,2,2020-08,2.0
153459,544135,ALPCOLCHR800,20.5,2,2020-08,1.0
153460,544135,ALPCOLCHR800,78.0,2,2020-08,1.0
153461,544135,ALPCOLCHR800,140.0,2,2020-08,1.0


#ABC analysis
For an effective ABC analysis, you have to take two things into account:

1.   The cost of the item
2.   How fast the item moves



In [ ]:
versa_sales_568

,id,part_number,price,currency_id,transaction_date,delta
1,153724,AAALPALIEBNONE,49.0,2,2019-10,1.0
2,153724,AAALPALIEBNONE,49.0,2,2020-01,1.0
4,153724,AAALPALIEBNONE,100.0,2,2019-10,1.0
5,153724,AAALPALIEBNONE,100.0,2,2020-01,1.0
7,153724,AAALPALIEBNONE,160.0,2,2019-10,1.0
...,...,...,...,...,...,...
153458,544134,ALPLIAPOP800,175.0,2,2020-08,2.0
153459,544135,ALPCOLCHR800,20.5,2,2020-08,1.0
153460,544135,ALPCOLCHR800,78.0,2,2020-08,1.0
153461,544135,ALPCOLCHR800,140.0,2,2020-08,1.0


In [ ]:
if (versa_sales_568['currency_id']==2).any():
    versa_sales_568['revenue'] = versa_sales_568.price * versa_sales_568.delta
elif (versa_sales_568['currency_id']==3).any():
     versa_sales_568['revenue'] = versa_sales_568.price * versa_sales_568.delta * 1.41

In [ ]:
    versa_sales_568['revenue'] = versa_sales_568.price * versa_sales_568.delta

In [ ]:
if (versa_sales_568['currency_id']==3).any():
    print(high)

KeyError: ignored

In [ ]:
versa_sales_568

,id,part_number,price,currency_id,transaction_date,delta,revenue
1,153724,AAALPALIEBNONE,49.0,2,2019-10,1.0,49.0
2,153724,AAALPALIEBNONE,49.0,2,2020-01,1.0,49.0
4,153724,AAALPALIEBNONE,100.0,2,2019-10,1.0,100.0
5,153724,AAALPALIEBNONE,100.0,2,2020-01,1.0,100.0
7,153724,AAALPALIEBNONE,160.0,2,2019-10,1.0,160.0
...,...,...,...,...,...,...,...
153458,544134,ALPLIAPOP800,175.0,2,2020-08,2.0,350.0
153459,544135,ALPCOLCHR800,20.5,2,2020-08,1.0,20.5
153460,544135,ALPCOLCHR800,78.0,2,2020-08,1.0,78.0
153461,544135,ALPCOLCHR800,140.0,2,2020-08,1.0,140.0


In [ ]:
versa_sales_568_abc = versa_sales_568.sort_values(by='revenue', ascending=False)

In [ ]:
n = len(versa_sales_568_abc)

In [ ]:
A, B, C = np.split(versa_sales_568_abc, [int(.2*n), int(.5*n)])

In [ ]:
A['revenue'].sum().astype(int)

7056980

In [ ]:
print(A, B, C)

            id      part_number  price  ...  transaction_date  delta  revenue
116001  385095     MAGMORGRYLRG  130.0  ...           2020-01  296.0  38480.0
115998  385095     MAGMORGRYLRG  105.6  ...           2020-01  296.0  31257.6
132037  477768  TRVCOTBRDWGR055   49.8  ...           2019-06  569.0  28336.2
116010  385096     MAGMORGRYMED   90.0  ...           2020-01  298.0  26820.0
116007  385096     MAGMORGRYMED   72.0  ...           2020-01  298.0  21456.0
...        ...              ...    ...  ...               ...    ...      ...
126980  469589     LINELILIM400  320.0  ...           2019-04    1.0    320.0
134504  487032     COTPERPOR500   64.0  ...           2019-08    5.0    320.0
147403  514087     LINELICHR400  320.0  ...           2020-03    1.0    320.0
127775  471061     LINSWEBLU400  160.0  ...           2019-04    2.0    320.0
31748   156588     LINLOUNAT465  320.0  ...           2020-03    1.0    320.0

[11895 rows x 7 columns]             id   part_number  price  .

#High, Medium and Low velocity items
After, ABC analysis for your inventory, but sometimes you have some really important items that will fall into the C category. That means we have to address the classification from another angle.

In [ ]:
 versa_sales_568_hml= versa_sales_568.sort_values(by='delta', ascending=False)
versa_sales_568_hml

,id,part_number,price,currency_id,transaction_date,delta,revenue
132037,477768,TRVCOTBRDWGR055,49.80,2,2019-06,569.0,28336.20
131492,476551,PLITRVFD1115,7.50,2,2019-09,569.0,4267.50
132036,477768,TRVCOTBRDWGR055,12.67,2,2019-06,569.0,7209.23
116004,385096,MAGMORGRYMED,45.00,2,2020-01,298.0,13410.00
116010,385096,MAGMORGRYMED,90.00,2,2020-01,298.0,26820.00
...,...,...,...,...,...,...,...
128669,471680,COTPERPOR340,116.00,2,2019-05,1.0,116.00
98161,352607,COTPERSLT340,116.00,2,2020-04,1.0,116.00
98162,352607,COTPERSLT340,116.00,2,2020-06,1.0,116.00
128670,471680,COTPERPOR340,116.00,2,2019-06,1.0,116.00


In [ ]:
n = len(versa_sales_568_hml)

In [ ]:
sum = versa_sales_568_hml.delta.sum()

In [ ]:
sum

137333.0

In [ ]:
high, medium, low = np.split(versa_sales_568_hml, [int(.2*n), int(.5*n)])

In [ ]:
print(high, medium, low)

            id      part_number  price  ...  transaction_date  delta   revenue
132037  477768  TRVCOTBRDWGR055  49.80  ...           2019-06  569.0  28336.20
131492  476551     PLITRVFD1115   7.50  ...           2019-09  569.0   4267.50
132036  477768  TRVCOTBRDWGR055  12.67  ...           2019-06  569.0   7209.23
116004  385096     MAGMORGRYMED  45.00  ...           2020-01  298.0  13410.00
116010  385096     MAGMORGRYMED  90.00  ...           2020-01  298.0  26820.00
...        ...              ...    ...  ...               ...    ...       ...
142326  502595     COTPERHON360  18.99  ...           2020-01    3.0     56.97
107892  374916     COTPERSTN360  18.99  ...           2020-03    3.0     56.97
59819   166658     ALPLIASPI800  78.00  ...           2020-03    3.0    234.00
98133   352607     COTPERSLT340  18.99  ...           2019-09    3.0     56.97
126093  468731     COTPERCEL360  18.99  ...           2019-12    3.0     56.97

[11895 rows x 7 columns]             id      part_n

In [ ]:
ha = pd.merge(high, A, how='inner')
hb = pd.merge(high, B, how='inner')
hc = pd.merge(high, C, how='inner')
ma = pd.merge(medium, A, how='inner')
mb = pd.merge(medium, B, how='inner')
mc = pd.merge(medium, C, how='inner')
la = pd.merge(low, A, how='inner')
lb = pd.merge(low, B, how='inner')
lc = pd.merge(low, C, how='inner')

In [ ]:
print(ha,hb,hc,ma,mb,mc,la,lb,lc)

          id      part_number   price  ...  transaction_date  delta   revenue
0     477768  TRVCOTBRDWGR055   49.80  ...           2019-06  569.0  28336.20
1     476551     PLITRVFD1115    7.50  ...           2019-09  569.0   4267.50
2     477768  TRVCOTBRDWGR055   12.67  ...           2019-06  569.0   7209.23
3     385096     MAGMORGRYMED   45.00  ...           2020-01  298.0  13410.00
4     385096     MAGMORGRYMED   90.00  ...           2020-01  298.0  26820.00
...      ...              ...     ...  ...               ...    ...       ...
6408  383345     COTJWLGRY345  148.00  ...           2019-05    3.0    444.00
6409  515689     LINVIVMIN500  135.00  ...           2020-03    3.0    405.00
6410  513257     LINCAMPNK500  135.00  ...           2020-03    3.0    405.00
6411  182360     LINEMIMIN520  135.00  ...           2020-03    3.0    405.00
6412  155246     COTMAYBLK300  135.00  ...           2019-03    3.0    405.00

[6413 rows x 7 columns]           id   part_number  price  curr

#Widgets

In [ ]:
widget = versa_sales[(versa_sales["firm_id"]==568)]

In [ ]:
widget = widget.groupby(['id', 'part_number', 'price'], as_index=False).sum()

In [ ]:
widget = widget.drop(['currency_id'], axis=1)

In [ ]:
widget.delta.describe()

count    89464.000000
mean         0.364549
std          3.055714
min        -13.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        268.000000
Name: delta, dtype: float64

In [ ]:
widget.sort_values(by='delta', ascending=False)

#Data Purity

# New Section

In [ ]:
# price negative or zero
zero_price = versa_sales[(versa_sales['price']==0) | (versa_sales['price'] <0)]

,name,part_number,price,id,firm_id,delta,transaction_date,currency_id,currencies_name
228,ER1000P BR,ER1000P BR,0.0,139326,567,-3.0,2017-02-07,2,United States dollar
229,ER325P BG,ER325P BG,0.0,139327,567,-3.0,2017-02-07,2,United States dollar
233,ER525P BR,ER525P BR,0.0,139332,567,-6.0,2017-02-07,2,United States dollar
249,SKSQ55P BR,SKSQ55P BR,0.0,139371,567,-15.0,2017-02-09,2,United States dollar
2275,"Tube-25ml Aqua,Decorated-Gentle Cleanser(KGCS25)",TB1-025KGCS25-01,0.0,165199,575,-8480.0,2017-08-25,9,Australian dollar
...,...,...,...,...,...,...,...,...,...
1795578,NaN,ME25644004,0.0,539950,581,14.0,2020-07-27,3,European Euro
1795714,Sample A.D. Laws Secale Straight Rye,ADL802234,0.0,498942,588,-1.0,2020-07-28,2,United States dollar
1795758,NaN,COTJULWHT700,0.0,513933,568,-3.0,2020-07-28,2,United States dollar
1795960,NaN,EUFILL-KO5,0.0,535567,575,-21199.0,2020-07-29,2,United States dollar
